In [4]:
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(dplyr))

args = commandArgs(trailingOnly = TRUE)
chr = 1


In [5]:
ref_df = as.data.frame(fread(paste0("/ysm-gpfs/pi/zhao-data/zy92/GTEx_V8/chr", chr, "_snp.txt")))
est_file_dir = "/gpfs/scratch60/zhao/zy92/GTEX/brain_pruned/ENET/chr"
est_file_list = list.files(paste0(est_file_dir, chr))
output_dir = paste0("/gpfs/scratch60/zhao/zy92/GTEX/weight_ENET/chr", chr)

if (!dir.exists(output_dir)) {
    dir.create(output_dir, recursive = T)
}
   
weight_list = list()
weight_snps = list()
construction_list = list()
sample_list = list()

gene_vec = c()

In [6]:
length(est_file_list)

[1] 2890

In [17]:


for (i in 14) {
    # read est
    #if (i %% 20 == 0) {
        print(paste0("INFO: gene ", i))
    #}
    gene = est_file_list[i]
    est_dir = paste0(est_file_dir, chr, "/", est_file_list[i])
    est_file = list.files(est_dir, pattern = "*_weights.txt")
    if (length(est_file) > 0) {   
        est_file = paste0(est_dir, "/", est_file)
        est_df = as.data.frame(fread(est_file))
        joint_df = est_df %>%
            inner_join(ref_df, by = c("id" = "variant_id")) %>%
            as.data.frame()
        
        
        tissue_vec = colnames(est_df)[7:ncol(est_df)]
        # for new tissue vec
        #tissue_vec = as.character(sapply(tissue_vec, function(x) unlist(strsplit(x, "\\."))[1]))
                                  
        # write est 

        for (tissue_idx in 1:length(tissue_vec)) {
            tissue = tissue_vec[tissue_idx]
            #IRdisplay::display_html(tissue)
            # weight table
            if (!tissue %in% names(weight_list)) {
                weight_list[[tissue]] = data.frame(rsid = joint_df$rs_id_dbSNP151_GRCh38p7,
                                                 gene = gene,
                                                 weight = joint_df[, tissue_idx + 6],
                                                 ref_allele = joint_df$ref.x,
                                                 eff_allele = joint_df$alt.x
                                                ) %>% 
                filter(weight != 0)
                    
                
            } else {
                tmp_df  = data.frame(rsid = joint_df$rs_id_dbSNP151_GRCh38p7,
                                                 gene = gene,
                                                 weight = joint_df[, tissue_idx + 6],
                                                 ref_allele = joint_df$ref.x,
                                                 eff_allele = joint_df$alt.x
                                     ) %>% 
                filter(weight != 0)
                
                weight_list[[tissue]] = rbind(weight_list[[tissue]], tmp_df)
            }
            
            # extra table
            if (!tissue %in% names(weight_snps)) {
                weight_snps[[tissue]] = data.frame(gene = gene,
                                                 genename = gene,
                                                 pred.perf.R2 = 0.5,
                                                 n.snps.in.model = nrow(joint_df),
                                                 pred.perf.pval = 1e-10,
                                                 pred.perf.qval = 1e-10
                                                )
                
            } else {
                tmp_df  = data.frame(gene = gene,
                                                 genename = gene,
                                                 pred.perf.R2 = 0.5,
                                                 n.snps.in.model = nrow(joint_df),
                                                 pred.perf.pval = 1e-10,
                                                 pred.perf.qval = 1e-10
                                                )
                                     
                weight_snps[[tissue]] = rbind(weight_snps[[tissue]], tmp_df)
            
           
                
            }
            
            # construction table
            construction_list[[tissue]] = data.frame(chr = 1:22, cv.seed = rep(1, 22))
            
            # sample table
            sample_list[[tissue]] = data.frame(n.samples = 500)
            
        }
        
    }
                   
                   
}



[1] "INFO: gene 14"


Warning message in fread(est_file):
“Detected 8 column names but the data has 7 columns. Filling rows automatically. Set fill=TRUE explicitly to avoid this warning.”


In [11]:
est_file


[1] "/gpfs/scratch60/zhao/zy92/GTEX/brain_pruned/ENET/chr1/ENSG00000007968.6/joint_weights.txt"

In [12]:
tmp_df

gene,genename,pred.perf.R2,n.snps.in.model,pred.perf.pval,pred.perf.qval
<fct>,<fct>,<dbl>,<int>,<dbl>,<dbl>
ENSG00000007968.6,ENSG00000007968.6,0.5,530,1e-10,1e-10


In [18]:
weight_list[[1]]

rsid,gene,weight,ref_allele,eff_allele
<fct>,<fct>,<dbl>,<fct>,<fct>
rs4908370,ENSG00000000938.12,-0.019349198,C,G
rs921517,ENSG00000000971.15,-0.003191778,A,T
rs138726843,ENSG00000001461.16,-0.029459334,T,G
rs35109721,ENSG00000001461.16,-0.011928068,C,T
rs7524076,ENSG00000001461.16,0.002687822,G,A
rs143210557,ENSG00000001461.16,-0.049315302,C,T
rs144883991,ENSG00000001461.16,0.007099638,G,C
rs115026711,ENSG00000001461.16,0.007103146,C,T
rs181862027,ENSG00000001461.16,-0.004010610,G,A


In [14]:
tmp_df

gene,genename,pred.perf.R2,n.snps.in.model,pred.perf.pval,pred.perf.qval
<fct>,<fct>,<dbl>,<int>,<dbl>,<dbl>
ENSG00000007933.12,ENSG00000007933.12,0.5,557,1e-10,1e-10


In [15]:
weight_list[[1]]

rsid,gene,weight,ref_allele,eff_allele
<fct>,<fct>,<dbl>,<fct>,<fct>
rs4908370,ENSG00000000938.12,-0.019349198,C,G
rs921517,ENSG00000000971.15,-0.003191778,A,T
rs138726843,ENSG00000001461.16,-0.029459334,T,G
rs35109721,ENSG00000001461.16,-0.011928068,C,T
rs7524076,ENSG00000001461.16,0.002687822,G,A
rs143210557,ENSG00000001461.16,-0.049315302,C,T
rs144883991,ENSG00000001461.16,0.007099638,G,C
rs115026711,ENSG00000001461.16,0.007103146,C,T
rs181862027,ENSG00000001461.16,-0.004010610,G,A
